# CDF Response Time Analysis with Different Service Distributions

This example demonstrates CDF response time analysis for a closed network with two classes and different service distributions. It compares results from Fluid solver and JMT transient analysis.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.fluid.*
import jline.solvers.jmt.*
import jline.util.matrix.*

In [ ]:
// CDF Response Time Analysis Example 4
// Closed network with two classes and different service distributions

val model = Network("model")

// Create closed queueing network
val delay = Delay(model, "Delay")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)

// Two closed classes with different populations
val class1 = ClosedClass(model, "Class1", 1, delay, 0)  // 1 job of Class1
val class2 = ClosedClass(model, "Class2", 3, delay, 0)  // 3 jobs of Class2

In [ ]:
// Service processes at Delay node
delay.setService(class1, Exp.fitMean(1.0))                    // Class1: Exp with mean=1.0
delay.setService(class2, Erlang.fitMeanAndOrder(4.0, 2))      // Class2: Erlang with mean=4.0, order=2

// Service processes at Queue1 node  
queue1.setService(class1, Exp.fitMean(2.0))                   // Class1: Exp with mean=2.0
queue1.setService(class2, HyperExp.fitMeanAndSCV(5.0, 30.0))  // Class2: HyperExp with mean=5.0, SCV=30.0

In [ ]:
// Routing matrix - independent circulation for each class
val P = model.initRoutingMatrix()

// Class 1 circular routing
P.set(class1, class1, delay, queue1, 1.0)
P.set(class1, class1, queue1, delay, 1.0)

// Class 2 circular routing 
P.set(class2, class2, delay, queue1, 1.0)
P.set(class2, class2, queue1, delay, 1.0)

model.link(P)

In [ ]:
// Solve with Fluid solver
val fluidSolver = FLD(model)
val rdFluid = fluidSolver.cdfRespT
println("RDfluid obtained")

In [ ]:
// Solve with JMT transient CDF analysis
val jmtOptions = JMT.defaultOptions()
jmtOptions.samples = 100000  // 100,000 samples
jmtOptions.seed = 23000

val jmtSolver = JMT(model, jmtOptions)
val rdSim = jmtSolver.tranCdfRespT
println("RDsim obtained")

In [ ]:
// Display results comparison
println("\n=== CDF Response Time Analysis Results ===")

for (i in 0 until model.numberOfStations) {
    println("\nNode $i (${model.stations[i].name}):")
    
    for (c in 0 until model.numberOfClasses) {
        val className = model.classes[c].name
        println("  Class $c ($className):")
        
        // Display fluid results sample
        if (rdFluid[i][c] != null) {
            val fluidData = rdFluid[i][c]
            println("    Fluid CDF points: ${fluidData.length()}")
            if (fluidData.length() > 0) {
                println("    Fluid range: [${fluidData.get(0, 1)}, ${fluidData.get(fluidData.length()-1, 1)}]")
            }
        }
        
        // Display simulation results sample
        if (rdSim[i][c] != null) {
            val simData = rdSim[i][c]
            println("    Simulation CDF points: ${simData.length()}")
            if (simData.length() > 0) {
                println("    Simulation range: [${simData.get(0, 1)}, ${simData.get(simData.length()-1, 1)}]")
            }
        }
    }
}

In [ ]:
// Calculate average response times from CDFs
val avgRespTFromCDFFluid = Matrix.zeros(model.numberOfStations, model.numberOfClasses)
val avgRespTFromCDFSim = Matrix.zeros(model.numberOfStations, model.numberOfClasses)

for (i in 0 until model.numberOfStations) {
    for (c in 0 until model.numberOfClasses) {
        // From Fluid CDF
        if (rdFluid[i][c] != null && rdFluid[i][c].length() > 1) {
            val fluidData = rdFluid[i][c]
            var sum = 0.0
            for (j in 1 until fluidData.length()) {
                val diff = fluidData.get(j, 0) - fluidData.get(j-1, 0)
                val value = fluidData.get(j, 1)
                sum += diff * value
            }
            avgRespTFromCDFFluid.set(i, c, sum)
        }
        
        // From Simulation CDF
        if (rdSim[i][c] != null && rdSim[i][c].length() > 1) {
            val simData = rdSim[i][c]
            var sum = 0.0
            for (j in 1 until simData.length()) {
                val diff = simData.get(j, 0) - simData.get(j-1, 0)
                val value = simData.get(j, 1)
                sum += diff * value
            }
            avgRespTFromCDFSim.set(i, c, sum)
        }
    }
}

println("\nAverage Response Times from CDF (Fluid):")
avgRespTFromCDFFluid.print()
println("\nAverage Response Times from CDF (Simulation):")
avgRespTFromCDFSim.print()

This example demonstrates:

1. **Multi-Class Closed Network**: Two classes with different populations (1 and 3 jobs)
2. **Different Service Distributions**: 
   - Exponential, Erlang, and Hyperexponential distributions
   - Different mean service times and variability (SCV) parameters
3. **CDF Analysis Methods**:
   - Fluid solver for steady-state analysis
   - JMT transient simulation for comparison
4. **Response Time Analysis**: Computation of average response times from CDFs

The comparison between fluid and simulation results helps validate the accuracy of different solution approaches for networks with non-exponential service distributions.